## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [3]:
import urllib.request
import zipfile
import lxml.etree

In [10]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [11]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

In [16]:
print(type(input_text))
print(input_text[:1000])

<class 'str'>
Here are two reasons companies fail: they only do more of the same, or they only do what's new.
To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation. Both are necessary, but it can be too much of a good thing.
Consider Facit. I'm actually old enough to remember them. Facit was a fantastic company. They were born deep in the Swedish forest, and they made the best mechanical calculators in the world. Everybody used them. And what did Facit do when the electronic calculator came along? They continued doing exactly the same. In six months, they went from maximum revenue ... and they were gone. Gone.
To me, the irony about the Facit story is hearing about the Facit engineers, who had bought cheap, small electronic calculators in Japan that they used to double-check their calculators.
(Laughter)
Facit did too much exploitation. But exploration can go wild, too.
A few years back, I worked closely alongsi

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [17]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [18]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [19]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [20]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [21]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [22]:
len(sentences_ted)

266694

In [23]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [4]:
from collections import Counter 

In [36]:
counts_ted_top1000 = []
c = Counter([word for sent in sentences_ted for word in sent])

In [38]:
list_most_common = c.most_common(1000)
print(list_most_common[:10])

[('the', 207748), ('and', 149305), ('to', 125169), ('of', 114818), ('a', 105399), ('that', 95146), ('i', 83180), ('in', 78070), ('it', 74738), ('you', 70923)]


In [44]:
for word, count in list_most_common:
    counts_ted_top1000.append(count)
    
print(counts_ted_top1000[:10])
print(counts_ted_top1000[-10:])

[207748, 149305, 125169, 114818, 105399, 95146, 83180, 78070, 74738, 70923]
[407, 407, 406, 406, 406, 405, 405, 404, 404, 404]


Plot distribution of top-1000 words

In [42]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [5]:
from gensim.models import Word2Vec

In [47]:
model = Word2Vec(sentences_ted, min_count=10, size=100)

In [48]:
model.save("ted_word2vec_model")

In [49]:
model_ted = Word2Vec.load("ted_word2vec_model")

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [50]:
model_ted.most_similar("man")

[('woman', 0.861563503742218),
 ('guy', 0.8053919672966003),
 ('lady', 0.7572652697563171),
 ('boy', 0.7567627429962158),
 ('girl', 0.7335348129272461),
 ('soldier', 0.7308832406997681),
 ('gentleman', 0.7243051528930664),
 ('poet', 0.6800374984741211),
 ('kid', 0.675588071346283),
 ('friend', 0.6711434125900269)]

In [51]:
model_ted.most_similar("computer")

[('machine', 0.7380391359329224),
 ('device', 0.7166091799736023),
 ('software', 0.713715672492981),
 ('robot', 0.6945363283157349),
 ('printer', 0.6483083963394165),
 ('3d', 0.6442069411277771),
 ('camera', 0.640776515007019),
 ('chip', 0.6384934186935425),
 ('interface', 0.6292914152145386),
 ('program', 0.6174736022949219)]

In [54]:
print(len(model_ted.wv.vocab))

14427


#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [55]:
words_top_ted = []
for word, count in list_most_common:
    words_top_ted.append(word)
    
print(words_top_ted[:10])

['the', 'and', 'to', 'of', 'a', 'that', 'i', 'in', 'it', 'you']


In [56]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

In [19]:
from sklearn.manifold import TSNE

In [57]:
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [58]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [6]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [7]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [8]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [9]:
print(type(sentences_wiki))
print(sentences_wiki[:10])

<class 'list'>
['   valkyria chronicles iii   ', ' senj  no valkyria     unrecorded chronicles   japanese                 lit ', ' valkyria of the battlefield       commonly referred to as valkyria chronicles iii outside japan   is a tactical role     playing video game developed by sega and media', 'vision for the playstation portable ', ' released in january      in japan   it is the third game in the valkyria series ', ' employing the same fusion of tactical and real     time gameplay as its predecessors   the story runs parallel to the first game and follows the   nameless     a penal military unit serving the nation of gallia during the second europan war who perform secret black operations and are pitted against the imperial unit   calamaty raven   ', ' the game began development in        carrying over a large portion of the work done on valkyria chronicles ii ', ' while it retained the standard features of the series   it also underwent multiple adjustments   such as making the

In [10]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


In [58]:
print(sentences_wiki[1])

temp = []
for sent in sentences_wiki:
    temp.append(sent.split())
    
print(len(temp))
print(temp[1])

 atheists and skeptics give speeches on various topics   and a debate with christian experts is held 
853422
['atheists', 'and', 'skeptics', 'give', 'speeches', 'on', 'various', 'topics', 'and', 'a', 'debate', 'with', 'christian', 'experts', 'is', 'held']


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [59]:
sentences_wiki = temp 

counts_wiki_top1000 = []
c_wiki = Counter([word for sent in sentences_wiki for word in sent if word != '' and word != ' '])

In [60]:
list_most_common_wiki = c_wiki.most_common(1000)

In [61]:
print(list_most_common_wiki[:1000])

[('the', 1283842), ('of', 546823), ('and', 500056), ('in', 434592), ('to', 398337), ('a', 358101), ('was', 214543), ('s', 164221), ('on', 152865), ('as', 145325), ('for', 139024), ('that', 137505), ('with', 129472), ('by', 126396), ('is', 102543), ('his', 93310), ('at', 92233), ('he', 91481), ('it', 89053), ('from', 88313), ('were', 70464), ('an', 64686), ('had', 58363), ('which', 55462), ('be', 47271), ('this', 43286), ('but', 42370), ('are', 41049), ('first', 40545), ('not', 39666), ('their', 38755), ('after', 38235), ('one', 37763), ('her', 37057), ('also', 36953), ('its', 36663), ('they', 34841), ('two', 34554), ('or', 33712), ('have', 32786), ('who', 31642), ('been', 30690), ('new', 30333), ('has', 30007), ('she', 29062), ('when', 27098), ('during', 26423), ('time', 26420), ('other', 25716), ('would', 25301), ('into', 24790), ('all', 24042), ('i', 23737), ('more', 23030), ('while', 21535), ('over', 21323), ('him', 20177), ('later', 19781), ('only', 19697), ('up', 19406), ('game', 

In [62]:
for word, count in list_most_common_wiki:
    counts_wiki_top1000.append(count)
    
print(counts_wiki_top1000[:100])
print(counts_wiki_top1000[-100:])

[1283842, 546823, 500056, 434592, 398337, 358101, 214543, 164221, 152865, 145325, 139024, 137505, 129472, 126396, 102543, 93310, 92233, 91481, 89053, 88313, 70464, 64686, 58363, 55462, 47271, 43286, 42370, 41049, 40545, 39666, 38755, 38235, 37763, 37057, 36953, 36663, 34841, 34554, 33712, 32786, 31642, 30690, 30333, 30007, 29062, 27098, 26423, 26420, 25716, 25301, 24790, 24042, 23737, 23030, 21535, 21323, 20177, 19781, 19697, 19406, 19383, 19007, 18991, 18923, 18700, 18688, 18498, 17987, 17961, 16939, 16891, 16525, 16184, 16080, 15666, 15616, 15615, 15406, 15315, 15100, 14781, 14652, 14644, 14077, 13978, 13865, 13853, 13803, 13698, 13675, 13621, 13594, 13539, 13388, 13336, 13271, 13243, 12973, 12814, 12811]
[2076, 2076, 2075, 2071, 2070, 2067, 2062, 2054, 2053, 2052, 2052, 2052, 2052, 2051, 2050, 2050, 2049, 2048, 2045, 2043, 2041, 2031, 2031, 2030, 2030, 2028, 2025, 2022, 2018, 2014, 2013, 2013, 2011, 2005, 2004, 2004, 2004, 2003, 2003, 2003, 2000, 1998, 1994, 1994, 1993, 1993, 1992, 

In [63]:
words_top_wiki = []
for word, count in list_most_common_wiki:
    words_top_wiki.append(word)
    
print(words_top_wiki[:100])

['the', 'of', 'and', 'in', 'to', 'a', 'was', 's', 'on', 'as', 'for', 'that', 'with', 'by', 'is', 'his', 'at', 'he', 'it', 'from', 'were', 'an', 'had', 'which', 'be', 'this', 'but', 'are', 'first', 'not', 'their', 'after', 'one', 'her', 'also', 'its', 'they', 'two', 'or', 'have', 'who', 'been', 'new', 'has', 'she', 'when', 'during', 'time', 'other', 'would', 'into', 'all', 'i', 'more', 'while', 'over', 'him', 'later', 'only', 'up', 'game', 'most', 'about', 'three', 'out', 'th', 'between', 'there', 'than', 'film', 'some', 'may', 'before', 'year', 'made', 'no', 'such', 'season', 'second', 'where', 'war', 'years', 'world', 'both', 'however', 'them', 'through', 'then', 'used', 'being', 'series', 'these', 'song', 'against', 'album', 'm', 'many', 'team', 'part', 'number']


In [64]:
model_wiki = Word2Vec(sentences_wiki, min_count=10, size=100)

In [65]:
model_wiki.save("wiki_word2vec_model")

In [66]:
model_wiki = Word2Vec.load("wiki_word2vec_model")

In [67]:
len(model_wiki.wv.vocab)

48688

#### t-SNE visualization

In [68]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [69]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)